In [ ]:
from collections import deque

# Function to read the cube from a text file
def read_cube(filename):
    cube = []
    start = None
    goal = None

    with open(filename, "r") as file:
        for i, line in enumerate(file):
            row = line.strip().split()
            if "S" in row:
                start = (i, row.index("S"))  # Find start position
                row[start[1]] = "0"  # Replace 'S' with 0
            if "G" in row:
                goal = (i, row.index("G"))  # Find goal position
                row[goal[1]] = "0"  # Replace 'G' with 0
            cube.append([int(cell) if cell.isdigit() else 0 for cell in row])  # Convert to integers
    return cube, start, goal

# BFS Algorithm
def bfs(cube, start, goal):
    rows, cols = len(cube), len(cube[0])
    queue = deque([(start, [start])])  # (current_position, path_taken)
    visited = set()

    while queue:
        (x, y), path = queue.popleft()
        if (x, y) == goal:
            return path  # Return the path if goal is reached

        if (x, y) in visited:
            continue
        visited.add((x, y))

        # Possible directions (Up, Down, Left, Right)
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and cube[nx][ny] == 0 and (nx, ny) not in visited:
                queue.append(((nx, ny), path + [(nx, ny)]))  # Add new position

    return -1  # No path found

# DFS Algorithm
def dfs(cube, start, goal):
    rows, cols = len(cube), len(cube[0])
    stack = [(start, [start])]
    visited = set()

    while stack:
        (x, y), path = stack.pop()
        if (x, y) == goal:
            return path

        if (x, y) in visited:
            continue
        visited.add((x, y))

        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < rows and 0 <= ny < cols and cube[nx][ny] == 0 and (nx, ny) not in visited:
                stack.append(((nx, ny), path + [(nx, ny)]))

    return -1

if __name__ == "__main__":
    filename = "cube.txt"
    cube, start, goal = read_cube(filename)

    print("Start Position:", start)
    print("Goal Position:", goal)

    bfs_path = bfs(cube, start, goal)
    dfs_path = dfs(cube, start, goal)

    print("\nBFS Path:", bfs_path if bfs_path != -1 else "No path found")
    print("DFS Path:", dfs_path if dfs_path != -1 else "No path found")


Start Position: (0, 2)
Goal Position: (6, 0)

BFS Path: [(0, 2), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4), (5, 4), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)]
DFS Path: [(0, 2), (0, 3), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5), (4, 4), (5, 4), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import heapq  # Priority queue for UCS

# Function to perform Uniform Cost Search (UCS)
def uniform_cost_search(graph, start, goal):
    priority_queue = [(0, start, [])]  # (cost, current_node, path_so_far)
    visited = set()  # Track visited nodes to avoid re-exploring them

    while priority_queue:
        cost, node, path = heapq.heappop(priority_queue)  # Get the node with the lowest cost

        if node in visited:
            continue  # Skip if already visited

        path = path + [node]  # Add the node to the current path
        visited.add(node)  # Mark as visited

        if node == goal:
            return path, cost  # Return path and total cost if goal reached

        for neighbor, weight in graph.get(node, []):
            if neighbor not in visited:
                heapq.heappush(priority_queue, (cost + weight, neighbor, path))

    return None, float("inf")  # Return None if no path exists


graph = {
    'A': [('B', 2), ('C', 1)],
    'B': [('A', 2), ('D', 4), ('C', 3)],
    'C': [('A', 1), ('B', 3), ('D', 2)],
    'D': [('B', 4), ('C', 2)]
}

# Start and Goal Nodes
start_node = 'A'
goal_node = 'D'

path, cost = uniform_cost_search(graph, start_node, goal_node)

if path:
    print("Shortest Path:", path)
    print("Total Cost:", cost)
else:
    print("No path found.")


Shortest Path: ['A', 'C', 'D']
Total Cost: 3


In [6]:
import heapq

# Read cube from file
def read_cube_from_file(filename):
    with open(filename, 'r') as file:
        cube = [line.strip().split() for line in file]
    return cube

# Heuristic function (Manhattan Distance)
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* Search Algorithm
def a_star_search(cube):
    rows, cols = len(cube), len(cube[0])

    # Find start and goal positions
    start, goal = None, None
    for r in range(rows):
        for c in range(cols):
            if cube[r][c] == 'S':  # Start position
                start = (r, c)
            elif cube[r][c] == 'G':  # Goal position
                goal = (r, c)

    if not start or not goal:
        return "Start or Goal not found!"

    # Priority queue (min-heap)
    pq = [(0, 0, start, [])]  # (total_cost, short_wall_count, (row, col), path_taken)
    visited = {}

    # Allowed movements (Up, Down, Left, Right)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    while pq:
        cost, short_wall_count, (r, c), path = heapq.heappop(pq)

        if (r, c) == goal:
            return path + [(r, c)], cost

        if (r, c) in visited and visited[(r, c)] <= short_wall_count:
            continue
        visited[(r, c)] = short_wall_count

        # Explore neighbors
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:  # Check within bounds
                cell_value = cube[nr][nc]

                if cell_value == '1':  # High wall - Cannot pass
                    continue
                elif cell_value == '2':  # Short wall - Can jump, increase cost
                    heapq.heappush(pq, (cost + 1 + heuristic((nr, nc), goal), short_wall_count + 1, (nr, nc), path + [(r, c)]))
                else:  # Open path
                    heapq.heappush(pq, (cost + heuristic((nr, nc), goal), short_wall_count, (nr, nc), path + [(r, c)]))

    return "No path found"

filename = "walls.txt"  # Cube stored in a text file
cube = read_cube_from_file(filename)
path, short_walls_used = a_star_search(cube)
print("Shortest Path:", path)
print("Short walls jumped:", short_walls_used)


Shortest Path: [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (5, 4), (6, 4), (6, 3), (6, 2), (6, 1), (6, 0)]
Short walls jumped: 50
